In [30]:
#https://www.cs.bham.ac.uk/~petitcz/files/IWSEC2013.pdf
#general alg explained well

import random
import time

#based on https://eprint.iacr.org/2015/310.pdf (page 3)
def genSM3(FF, A, B):
    PR.<x1, x2, x3> = PolynomialRing(FF, 3, order='degrevlex')
    sm = (x1 - x2)^2*x3^2-2*((x1 + x2)*(x1*x2 + A) + 2*B)*x3 + (x1*x2 - A)^2 - 4*B*(x1 + x2)
    return sm

#random Elliptic curve over a field GF(p), p is a prime
#primeOrd forces returned EC to be of prime order
#log[2, p] ~= bits
#returns E(GF(p)) Finite Field and 3-rd summation polynomial for this curve
def generateRandomECandSM3(bits, primeOrd=False):
    p = next_prime(2**bits);
    T = GF(p)
    
    #general Weierstrass: Y^2 + a1*XY + a3*Y = X^3 + a2*X^2 + a4*X + a6

    #here we can use short Weierstrass eq char(T) = p
    #Y^2 = X^3 + A*X + B, A, B \in GF(p), 
    
    coefs = [0, 0, 0, 1, 1]
    while True:
        #random A
        coefs[3] = T.random_element();
        #random B
        coefs[4] = T.random_element();
        try:
            E = EllipticCurve(T, coefs)
            if primeOrd == False or is_prime(E.order()):
                break
        except ArithmeticError: #if E singular
            pass
    print(E)
    return E, T, genSM3(T, coefs[3], coefs[4])

#returns a random multiple of a base point P and order of subgroup <P>
def generateRandomPoint(P):
    orderP=P.order()
    k=int(1+(orderP-1)*random.random()) #k in [1, orderP-1]
    Q=k*P
    return Q, orderP

def buildFactorBaseOnlyX(Q, P, orderP, FF, m = 3):
    factorBaseSize = ceil(orderP^(1/m)) 
    factorBase = []
    coord = []
    
    currentSize = 0
    while currentSize < factorBaseSize:
        a = int(random.random()*orderP) #a,b in [0, orderE - 1]
        b = int(random.random()*orderP)
        candidate=(a*P+b*Q)[0] #we are only interested in the x-coordinate
        if candidate == 0: #identity
            print("found candidate: Identity: a={}, b={}.".format(a,b))
        elif candidate not in factorBase:
            factorBase.append(candidate)
            coord.append((a,b))
            currentSize += 1
        #else: TODO - can we solve ECDLP directly here?
    return coord, factorBase


#based on (Out Algorithm) http://www.science.unitn.it/~sala/events2016/AlessandroAmadori-BunnyTN7.pdf
def solveECDLP(E, SM3, P, Q, FF, orderP, coord, factorBase):
    PR.<U> = PolynomialRing(FF,1, order='degrevlex') 
    res = -1
    factorBaseSize = len(factorBase)
    
    ident = E(0) #EC identity element
    
    #generators should be: and SM3(X1, X2, X3)
    for i in srange(0, factorBaseSize):
        for j in srange(i, factorBaseSize):
            for k in srange(j, factorBaseSize):
                #solve
#                tmp = PR.ideal(generators);
 #               gb = tmp.groebner_basis('libsingular:groebner')
                if (SM3(factorBase[i], factorBase[j], factorBase[k]) == 0):
                    #there exist some points: 
                    # +-(X1, y1) +- (X2, y2) +- (U, Uy) = identity(E) = +-(X2, y2) +- (X3, y3) +- (U, Uy)
                    #use coords to solve
                    points = [E.lift_x(factorBase[i]), E.lift_x(factorBase[j]), E.lift_x(factorBase[k])]
                    #print(points)
                    for v in VectorSpace(GF(2), 3):
                        test = sum(-points[k] if v[k] else points[k] for k in [0,1,2])
                        if test == ident:
                            sumA = 0
                            sumB = 0
                            for k in [0,1,2]:
                                baseId = factorBase.index(points[k][0])
                                if ( (coord[baseId][0]*P + coord[baseId][1]*Q) == (-1)^(v[k])*points[k] ):
                                    sumA -= coord[baseId][0]
                                    sumB += coord[baseId][1]
                                else: 
                                    sumA += coord[baseId][0]
                                    sumB -= coord[baseId][1]
                            try:
                                res = Integer(mod(sumA*inverse_mod(sumB, orderP), orderP))
                                return res
                            except (ZeroDivisionError ,NotImplementedError): #inverse of 'sumB' mod 'orderP' does not exist
                                print("SumB = {}, SumA = {}, not invertible mod orderP ({}).".format(sumB, sumA, orderP))
                                pass
                        
    return res

def findPoints(xs, E):
    relation = []
    points = []
    ident = E(0) #infinity point (identity)
    xs = [xs["x1"], xs["x2"], xs["x3"]]
    for val in xs:
        if E.is_x_coord(val): #valid point
            adept = E.lift_x(val, all=False, extend=False)
            if adept != ident: #no new information
                points.append(adept)
    #find signs
    dim = len(points)
    if dim > 1:
        for v in VectorSpace(GF(2), dim - 1):
            if (points[-1] + sum(-points[k] if v[k] else points[k] for k in range(0, dim - 1))) == ident:
                relation = [-points[k] if v[k] else points[k] for k in range(0, dim - 1)]
                relation.append(points[-1])
                break
    return relation


#based on (Out Algorithm) http://www.science.unitn.it/~sala/events2016/AlessandroAmadori-BunnyTN7.pdf
def solveECDLP2(E, SM3, P, Q, FF, orderP, coord, factorBase):
    PR.<x1, x2, x3> = PolynomialRing(FF,3, order='degrevlex')
    PR1.<X> = PolynomialRing(FF,1, order='degrevlex')
    res = -1
    factorBaseSize = len(factorBase)
    
    ident = E(0) #EC identity element
    
    #generators should be: SM3(x1, x2, x3)
    #we can add additional contrains (instead of 3 cycles):
    #product(X - X1) = 0 => X1 is in the factorBase
    #product(X - X2) = 0 => X2 is in the factorBase
    #product(X - X3) = 0 => X3 is in the factorBase
    generators = [SM3(x1,x2,x3)]
    
    genPoly = product((X - factorBase[k]) for k in srange(0, factorBaseSize))
    generators.append(genPoly(x1))
    generators.append(genPoly(x2))
    generators.append(genPoly(x3))
    #solve
    tmp = PR.ideal(generators);
    gb = tmp.groebner_basis('libsingular:groebner')
    variety = (PR.ideal(gb)).variety()
    for solution in variety:
        relation = findPoints(solution, E)
        sumA = 0
        sumB = 0
        for rel in relation:
            if rel[0] in factorBase:
                baseId = factorBase.index(rel[0])
                if ( (coord[baseId][0]*P + coord[baseId][1]*Q) == rel ):
                    sumA -= coord[baseId][0]
                    sumB += coord[baseId][1]
                else: # (-rel) is in factorBase
                    sumA += coord[baseId][0]
                    sumB -= coord[baseId][1]
            else:
                print("something fishy...")
        try:
            res = Integer(mod(sumA*inverse_mod(sumB, orderP), orderP))
            if res*P == Q: #is it really the solution?
                break
            else:
                print("Found {} is not really a solution, something went wrong.".format(res))
        except (ZeroDivisionError ,NotImplementedError): #inverse of 'sumB' mod 'orderP' does not exist
            print("SumB = {}, SumA = {}, not invertible mod orderP ({}).".format(sumB, sumA, orderP))
            pass
                        
    return res

In [2]:
#### Tests:
bits = 17
E, FF, SM3 = generateRandomECandSM3(bits)
P = E.gen(0) #base point P
Q, orderP = generateRandomPoint(P)

dl = discrete_log(Q,P,operation="+")
print("result: {}".format(dl))

print("Order of P is {}.".format(orderP))

print("Factorization of Elliptic curve order: {} = {}".format(orderP, orderP.factor()))
sys.stdout.flush()

maxAttempts = 5
good = 0
tm = time.time()
for attempt in range(0, maxAttempts):
    coord, factorBase = buildFactorBaseOnlyX(Q, P, orderP, FF)
    print("Factorization base of lenght {} built.".format(len(factorBase)))
    sys.stdout.flush()
        
    res = solveECDLP2(E, SM3, P, Q, FF, orderP, coord, factorBase)
    print("Attempt {}, result: {}, res: {}.\n\n".format(attempt, bool(res*P == Q), res))
    sys.stdout.flush()
    
    good += int(res*P == Q)
print("(ECDLP2) Success rate: {}/{} it took {} seconds.".format(good, maxAttempts, time.time() - tm))


good = 0
tm = time.time()
for attempt in range(0, maxAttempts):
    coord, factorBase = buildFactorBaseOnlyX(Q, P, orderP, FF)
    print("Factorization base of lenght {} built.".format(len(factorBase)))
    sys.stdout.flush()
        
    res = solveECDLP(E, SM3, P, Q, FF, orderP, coord, factorBase)
    print("Attempt {}, result: {}, res: {}.\n\n".format(attempt, bool(res*P == Q), res))
    sys.stdout.flush()
    
    good += int(res*P == Q)
print("(ECDLP) Success rate: {}/{} it took {} seconds.".format(good, maxAttempts, time.time() - tm))


Elliptic Curve defined by y^2 = x^3 + 34941*x + 47114 over Finite Field of size 131101
result: 7157
Order of P is 32808.
Factorization of Elliptic curve order: 32808 = 2^3 * 3 * 1367
Factorization base of lenght 33 built.
Attempt 0, result: False, res: -1.


Factorization base of lenght 33 built.
SumB = 8198, SumA = -20426, not invertible mod orderP (32808).
SumB = 8198, SumA = -20426, not invertible mod orderP (32808).
SumB = -8198, SumA = 20426, not invertible mod orderP (32808).
SumB = -8198, SumA = 20426, not invertible mod orderP (32808).
SumB = 8198, SumA = -20426, not invertible mod orderP (32808).
SumB = 8198, SumA = -20426, not invertible mod orderP (32808).
Attempt 1, result: False, res: -1.


Factorization base of lenght 33 built.
Attempt 2, result: False, res: -1.


Factorization base of lenght 33 built.
Attempt 3, result: True, res: 7157.


Factorization base of lenght 33 built.
SumB = -17984, SumA = 27104, not invertible mod orderP (32808).
SumB = -17984, SumA = 27104, no

In [ ]:
#Individual step of Pohlig-Hellman algorithm
#Solves DLP in a subgroup of order 'orderP'
#https://courses.fit.cvut.cz/MI-MKY/media/lectures/mi-mky-poznamky-v17.pdf (page 81 PDF)
#Pi - generator of the subgroup, Qi = ki*Pi, order of Pi is orderP, returns ki
#Splits ki = x0 + p*x1 + p^2*x2 + p^(e-1)*x_(e-1) and solves 'e'-times DLP in a subgroup of order 'orderP'
def PH_reduction(Qi, Pi, orderP, **kwargs): 
    invPi = -Pi #additive inverse
    p = prime_factors(orderP)[0]
    e = orderP.valuation(p)
    
    Pi = (p^(e-1))*Pi
    ki = 0 #ki = x0 + p*x1 + p^2*x2 + ...
    for i in srange(0, e):
        tempQ = (p^(e-1-i))*(Qi + ki*invPi)
        ki += (p^i)*discrete_log_rho(tempQ, Pi, operation="+")
    return ki


#Implements Pohlig-Hellman algorithm
#https://courses.fit.cvut.cz/MI-MKY/media/lectures/mi-mky-poznamky-v17.pdf (page 81 PDF)
#Solves Q = k*P, returns k
#P generates a subgroup of order orderP
#DLP_Solve is a function solving ECDLP in a subgroup of order p^e, where 'p' is a prime number
def Pohlig_Hellman_additive(Q, P, orderP, DLP_Solve = PH_reduction, **kwargs):
    print("Factorization of orderP: {} = {}.".format(orderP, orderP.factor()))
    print("Order of E = <P> is {}.\nBit security (log2 of the biggest prime_factor of orderP): {:4.1f}"\
      .format(orderP, float(log(max(prime_factors(orderP)))/log2)))
    
    ki = [] #list of individual DLP results
    mi = [] #list of moduli
    for p in prime_factors(orderP): #prime factor p
        e = orderP.valuation(p) #multiplicity of p in orderP
        mi.append(p^e) 
        pei = Integer(orderP/mi[-1])
        ki.append(DLP_Solve(pei*Q, pei*P, mi[-1], **kwargs)) #solves ECDLP in a subgroup <pei*P>
    return Integer(CRT(ki, mi))

Elliptic Curve defined by y^2 = x^3 + 681*x + 1543 over Finite Field of size 2053
Finite Field of size 2053
2080
(12.7095346590463, 12.7650085977198)
